# KNN

[Ciclo 02: Aprendizado supervisionado - Classificação - KNN](https://membro.comunidadedatascience.com/89193-fundamentos-de-machine-learning/2150255-aula-11-k-nearest-neighbors-teoria)

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler